# Project "Know_Your_Rent_Muc": Part One
# Web Scraping

For the projet "Know_Your_Rent_Muc" it is importent to have dato to feed the machine learning algortihm. To get this data I write a program to "scrap" the data from a popular website with the latest information about rentable flats. The programm should screp all inforamtion of all flat ads of a website. It should iterate through all pages of the website.

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
from urllib import request
import pandas as pd
import io

First of all I get the connetction to the website.

In [2]:
webpage_response = requests.get("https://www.immonet.de/immobiliensuche/sel.do?parentcat=1&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city=121673&marketingtype=2&page=25")

After that I get the HTML code of the website.

In [3]:
webpage = webpage_response.content

Then I transform the html code to format which can is workable in python.

In [4]:
soup = BeautifulSoup(webpage, 'html.parser')

The information of my desire is at 8th place in the list with the tag "script"

In [5]:
s = soup.find_all("script")[8].text

In [6]:
s

'\n\t\t    var utag_data = {"enh_impression_id":["500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500"],"enh_impression_list":["Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend"],"enh_impression_name":["kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","ko

Because the format of the data is not ideal I need transform it so long unitl it´s a perfect dictionary. With this dictionary it is possible to transform it to a dataframe with Pandas. The dataframe is the format to feed my machine learning alogrithm. 

At first the messy string need to be preprocced. In this case I select just the date inside the {}.

In [7]:
def get_right_string(s):
    count1 = 0
    count2 = 0
    string = "{"
    for i in s:
        if i == "{":
            count1 += 1
        elif i == "}":
            count2 += 1
        elif count1 == 1 and count2 != 1:
            string += i
    string += "}"
    return string          

In [8]:
get_right_string(s)

'{"enh_impression_id":["500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500"],"enh_impression_list":["Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend"],"enh_impression_name":["kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt"],"enh_imp

In [9]:
s = get_right_string(s)

Now I split the data the colons to separate the keys and the values.

In [10]:
s = s.split(":")

In [11]:
s

['{"enh_impression_id"',
 '["500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500","500"],"enh_impression_list"',
 '["Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend"],"enh_impression_name"',
 '["kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","konta

The keys and values are not perfectly separated. Thats I need a function to find the remaining keys in the values. The following function find the strings which have both, keys and values. After that the positions where the string of the values ends and the key starts. 

In [12]:
def find_separation(string):
    position_list = -1
    pos_dict = {}
    count = 0
    for i in string:
        position_str = -1
        position_list += 1
        for j in i:
            position_str += 1
            if j == "[" or j == "]":
                count += 1
            elif count == 2 and j == ",":
                count = 0
                pos_dict[position_list] = position_str
                break
    return pos_dict

In [13]:
find_separation(s)

{1: 139,
 2: 553,
 3: 231,
 4: 61,
 5: 2,
 6: 2,
 7: 10,
 8: 2,
 14: 2,
 15: 2,
 22: 173,
 23: 131,
 24: 311,
 25: 139,
 26: 254,
 27: 222,
 28: 267,
 29: 208,
 30: 898,
 31: 70,
 32: 139,
 33: 185,
 34: 208,
 35: 277,
 36: 70,
 37: 254,
 38: 158,
 39: 94,
 40: 254}

The following piece of code separates finally the keys and values and save it in a dictonary. 

In [14]:
def separation(dict_, list_):
    dict_features = {}
    count = -1
    for i in dict_:
        if i == 1:
            save_last_i = i
        else:
            s = list_[save_last_i]
            e = dict_[save_last_i]
            k = s[e+1:]
            dict_features[k] = list_[i][:dict_[i]]
            save_last_i = i
    return dict_features

In [15]:
separation(find_separation(s), s)

{'"enh_impression_list"': '["Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend","Liste_Objekte_passend"]',
 '"enh_impression_name"': '["kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt","kontakt"]',
 '"enh_impression_position"': '[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]',
 '"search_locationnames"': '[]',
 '"search_districts"': '[]',

But now the values are just single strings. The following code transforms the single strings of each key to separate values and save it in a list. 

In [16]:
def transform_str(dict_):
    for i in dict_:
        list_ = []
        string = ""
        for j in dict_[i]:
            if j == "[" or j == '"':
                next
            elif j == ",":
                list_.append(string)
                string = ""
            elif j == "]":
                dict_[i] = list_
                list_ = []
            else:
                string += j
    return dict_

In [17]:
transform_str(separation(find_separation(s), s))

{'"enh_impression_list"': ['Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend'],
 '"enh_impression_name"': ['kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt'],
 '"enh_impression_position"': ['1',
  '2',
  '3',
  '4',
  '5',
  '6

Now the problem is that there are values which are actually numbers. The next function transform this strings to numbers.

In [18]:
def str_to_float(dict_):
    for i in dict_:
        k = len(dict_[i])
        for j in range(k):
            try:
                if dict_[i][j][0].isnumeric():
                    dict_[i][j] = float(dict_[i][j])
            except:
                next 
    return dict_

In [19]:
str_to_float(transform_str(separation(find_separation(s), s)))

{'"enh_impression_list"': ['Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend'],
 '"enh_impression_name"': ['kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt'],
 '"enh_impression_position"': [1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.

Before I can finally transform the dictonary to dataframe I need to select just the keys which have the same length and delete the others. To do that I just search for the values which are suspiciously short and delete them. 

In [20]:
keys = str_to_float(transform_str(separation(find_separation(s), s))).keys()

In [21]:
def search_same_length(dict_, keys):
    for i in keys:
        if len(dict_[i]) < 2:
            del dict_[i]
    return dict_

In [22]:
search_same_length(str_to_float(transform_str(separation(find_separation(s), s))), keys)

{'"enh_impression_list"': ['Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend',
  'Liste_Objekte_passend'],
 '"enh_impression_name"': ['kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt',
  'kontakt'],
 '"enh_impression_position"': [1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.

Finally I can transform the dictonary to a dataframe.

In [23]:
df = pd.DataFrame(search_same_length(str_to_float(transform_str(separation(find_separation(s), s))), keys))

In [24]:
df.head()

,"""enh_impression_list""","""enh_impression_name""","""enh_impression_position""","""search_sortby""","""object_area""","""object_city""","""object_currency""","""object_display_state""","""object_district""","""object_features""",...,"""object_is_special""","""object_label""","""object_marketingtype""","""object_objekt_nutzungsart""","""object_objekt_zustand""","""object_objektart""","""object_objektart_sub""","""object_price""","""object_rooms""","""object_listingtype"""
0,Liste_Objekte_passend,kontakt,1.0,1924332.0,298.00,Hohenbrunn,EUR,rec_item,null,EBK,...,,old,Miete,WOHNEN,sonstiges,,DACHGESCHOSS,6400.0,11.0,standard
1,Liste_Objekte_passend,kontakt,2.0,1695354.0,73.11,Taufkirchen,EUR,rec_item,Bergham,Garten|EBK,...,,old,Miete,WOHNEN,sonstiges,,DACHGESCHOSS,1330.0,2.0,standard
2,Liste_Objekte_passend,kontakt,3.0,334306.0,81.26,Taufkirchen,EUR,rec_item,Westerham,Garten|EBK,...,,old,Miete,WOHNEN,sonstiges,,ETAGE,1340.0,3.0,standard
3,Liste_Objekte_passend,kontakt,4.0,1695354.0,111.14,Taufkirchen,EUR,rec_item,Bergham,Garten|Balkon|EBK,...,,old,Miete,WOHNEN,sonstiges,,Wohnung,2090.0,4.0,standard
4,Liste_Objekte_passend,kontakt,5.0,334306.0,99.40,Taufkirchen,EUR,rec_item,Bergham,EBK,...,,old,Miete,WOHNEN,sonstiges,,Wohnung,1770.0,3.0,standard


This step is more relevant in the final step of this project. But in the stage of the web scraping it makes sense to this here. 
For the finale website it is easier for the customer to type the post code then the name of the district. That´s why I scrap the names of district with the matching post codes. 

Like in the previouse steps I get the html code from a website with the post code and convert in a proper fit for the searching for the needed informations.

In [25]:
webpage_res = requests.get("https://www.muenchen.de/leben/service/postleitzahlen-muenchen-die-plz-suche-nach-stadtteilen")

In [26]:
webpage_plz = webpage_res.content

In [27]:
soup_ = BeautifulSoup(webpage_plz, 'html.parser')

With the following code the needed informations get separated.

In [28]:
def select_plz(soup_):
    dict_ = {}
    keys = []
    for i in range(len(soup_.find_all("td", "class"=="district"))):
        if i == 0 or i%2 == 0:
            dict_[soup_.find_all("td", "class"=="district")[i].text] = 0
            keys.append(soup_.find_all("td", "class"=="district")[i].text)
    count = 0
    for i in range(len(soup_.find_all("td", "class"=="district"))):
        if i%2 != 0:
            dict_[keys[count]] = soup_.find_all("td", "class"=="district")[i].text
            count += 1
    return dict_

In [29]:
select_plz(soup_)

{'Allach-Untermenzing': '80995, 80997, 80999, 81247, 81249',
 'Altstadt-Lehel': '80331, 80333, 80335, 80336, 80469, 80538, 80539',
 'Au-Haidhausen': '81541, 81543, 81667, 81669, 81671, 81675, 81677',
 'Aubing-Lochhausen-Langwied': '81243, 81245, 81248, 81249',
 'Berg am Laim': '81671, 81673, 81735, 81825',
 'Bogenhausen': '81675, 81677, 81679, 81925, 81927, 81929',
 'Feldmoching-Hasenbergl': '80933, 80935, 80995',
 'Hadern': '80689, 81375, 81377',
 'Laim': '80686, 80687, 80689',
 'Ludwigsvorstadt-Isarvorstadt': '80335, 80336, 80337, 80469',
 'Maxvorstadt': '80333, 80335, 80539, 80636, 80797, 80798, 80799, 80801, 80802',
 'Milbertshofen-Am Hart': '80807, 80809, 80937, 80939',
 'Moosach': '80637, 80638, 80992, 80993, 80997',
 'Neuhausen-Nymphenburg': '80634, 80636, 80637, 80638, 80639',
 'Obergiesing': '81539, 81541, 81547, 81549',
 'Pasing-Obermenzing': '80687, 80689, 81241, 81243, 81245, 81247',
 'Ramersdorf-Perlach': '81539, 81549, 81669, 81671, 81735, 81737, 81739',
 'Schwabing-Freim

Because the post codes are all strings it is importent to convert them to integers. The format of this string from the website is a little bit different. That why the transform function need to be changed.

In [30]:
def transform_str2(dict_):
    for i in dict_:
        list_ = []
        string = ""
        for j in dict_[i]:
            if j == ",":
                list_.append(string)
                string = ""
            elif j == " ":
                next
            else:
                string += j
        dict_[i] = list_
    return dict_

The same count to the transformation function from string to numeric. 

In [31]:
def str_to_float2(dict_):
    for i in dict_:
        count = -1
        for j in dict_[i]:
            count += 1
            if j.isnumeric():
                dict_[i][count] = int(dict_[i][count])
    return dict_

In [32]:
str_to_float2(transform_str2(select_plz(soup_)))

{'Allach-Untermenzing': [80995, 80997, 80999, 81247],
 'Altstadt-Lehel': [80331, 80333, 80335, 80336, 80469, 80538],
 'Au-Haidhausen': [81541, 81543, 81667, 81669, 81671, 81675],
 'Aubing-Lochhausen-Langwied': [81243, 81245, 81248],
 'Berg am Laim': [81671, 81673, 81735],
 'Bogenhausen': [81675, 81677, 81679, 81925, 81927],
 'Feldmoching-Hasenbergl': [80933, 80935],
 'Hadern': [80689, 81375],
 'Laim': [80686, 80687],
 'Ludwigsvorstadt-Isarvorstadt': [80335, 80336, 80337],
 'Maxvorstadt': [80333, 80335, 80539, 80636, 80797, 80798, 80799, 80801],
 'Milbertshofen-Am Hart': [80807, 80809, 80937],
 'Moosach': [80637, 80638, 80992, 80993],
 'Neuhausen-Nymphenburg': [80634, 80636, 80637, 80638],
 'Obergiesing': [81539, 81541, 81547],
 'Pasing-Obermenzing': [80687, 80689, 81241, 81243, 81245],
 'Ramersdorf-Perlach': [81539, 81549, 81669, 81671, 81735, 81737],
 'Schwabing-Freimann': [80538, 80801, 80802, 80803, 80804, 80805, 80807],
 'Schwabing-West': [80796, 80797, 80798, 80799, 80801, 80803, 

The last part of this program is to write a function which iterate through all pages of the website, get the date of each page and convert them with the previous function to a dataframe. 

In [33]:
url = "https://www.immonet.de/immobiliensuche/sel.do?parentcat=1&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city=121673&marketingtype=2&page="

In [47]:
def webscraping(url):
    dict__= {}
    for i in range(1, 50):
        if i == 1:
            i = str(i)
            url_1 = url+i
            webpage_response = requests.get(url_1)
            webpage = webpage_response.content
            soup = BeautifulSoup(webpage, 'html.parser')
            s = soup.find_all("script")[8].text
            s = get_right_string(s)
            if len(s) > 4000:
                s = s.split(":")
                keys = str_to_float(transform_str(separation(find_separation(s), s))).keys()
                dict_ = search_same_length(str_to_float(transform_str(separation(find_separation(s), s))), keys) 
                dict_keys = dict_.keys()
                for j in dict_keys:
                    dict__[j] = dict_[j]
        else:
            i = str(i)
            url_1 = url+i
            webpage_response = requests.get(url_1)
            webpage = webpage_response.content
            soup = BeautifulSoup(webpage, 'html.parser')
            s = soup.find_all("script")[8].text
            s = get_right_string(s)
            if len(s) > 4000:
                s = s.split(":")
                keys = str_to_float(transform_str(separation(find_separation(s), s))).keys()
                dict_ = search_same_length(str_to_float(transform_str(separation(find_separation(s), s))), keys) 
                dict_keys = dict_.keys()
                for j in dict_keys:
                    dict__[j].extend(dict_[j])
    
    #Search for values of keys which are long then the other. Shorten this values.
    len_dict = []
    unequal = []
    for k in dict__:
        len_dict.append(len(dict__[k]))
    min_len = min(len_dict)
    for m in dict__:
        if len(dict__[m]) != min_len:
            dict__[m] = dict__[m][:min_len]   
    
    #Convert the dictonary to a dataframe.
    count = 0
    for z in dict__:
        if count == 0:
            count += 1
            df = pd.DataFrame(dict__[z], columns=[z])
        else:
            count += 1
            df[z] = dict__[z]
    return df  

In [48]:
df = webscraping(url)

Finally all the data were converted in a dataframe.

In [51]:
df.head()

,"""enh_impression_list""","""enh_impression_name""","""enh_impression_position""","""search_sortby""","""object_area""","""object_city""","""object_currency""","""object_display_state""","""object_district""","""object_features""",...,"""object_is_special""","""object_label""","""object_marketingtype""","""object_objekt_nutzungsart""","""object_objekt_zustand""","""object_objektart""","""object_objektart_sub""","""object_price""","""object_rooms""","""object_listingtype"""
0,Liste,kontakt,1.0,2594468.0,37.00,München,EUR,sel_item,Thalk.Obersendl.-Forsten-Fürstenr.-Solln,Balkon|EBK,...,,new,Miete,WOHNEN,sonstiges,,APARTMENT,1120.00,1.0,standard
1,Liste,kontakt,2.0,177005.0,53.06,München,EUR,sel_item,Thalk.Obersendl.-Forsten-Fürstenr.-Solln,Balkon,...,top-immo,old,Miete,WOHNEN,sonstiges,,ETAGE,1339.77,2.0,premium
2,Liste,kontakt,3.0,177005.0,64.16,München,EUR,sel_item,Thalk.Obersendl.-Forsten-Fürstenr.-Solln,,...,top-immo,old,Miete,WOHNEN,sonstiges,,ETAGE,1509.68,2.0,premium
3,Liste,kontakt,4.0,177005.0,49.46,München,EUR,sel_item,Thalk.Obersendl.-Forsten-Fürstenr.-Solln,,...,top-immo,old,Miete,WOHNEN,sonstiges,,ERDGESCHOSS,1600.03,2.0,premium
4,Liste,kontakt,5.0,177005.0,96.23,München,EUR,sel_item,Thalk.Obersendl.-Forsten-Fürstenr.-Solln,Balkon,...,top-immo,old,Miete,WOHNEN,sonstiges,,ETAGE,2085.30,4.0,premium


A short check of the shape helps to find errors in the convertation. For example, if there are to few rows this is a signal that something is wrong.

In [53]:
df.shape

(925, 22)

Everything is ok with the raw data. Know I convert it to a CSV. In the next part teh raw data will be preprocced for the machine learning algorithm.

In [50]:
df.to_csv('221211_know_your_rent_muc_data.csv')